In [5]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import mobilenet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import adam_v2 


In [7]:
base_model=mobilenet(weights='imagenet',include_top=False) 
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(3,activation='softmax')(x) 


TypeError: 'module' object is not callable

In [5]:
model=Model(inputs=base_model.input,outputs=preds)

In [6]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [8]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) 

train_generator=train_datagen.flow_from_directory('./train/', target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 197 images belonging to 3 classes.


In [9]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
step_size_train=train_generator.n//20
model.fit_generator(generator=train_generator,steps_per_epoch=step_size_train,epochs=5)

Epoch 1/5
9/9 [==============================] - 39s 4s/step - loss: 0.9498 - accuracy: 0.7318
Epoch 2/5
9/9 [==============================] - 33s 4s/step - loss: 0.1172 - accuracy: 0.9540
Epoch 3/5
9/9 [==============================] - 30s 3s/step - loss: 0.2023 - accuracy: 0.9573
Epoch 4/5
9/9 [==============================] - 33s 4s/step - loss: 0.1051 - accuracy: 0.9693
Epoch 5/5
9/9 [==============================] - 33s 4s/step - loss: 0.1243 - accuracy: 0.9655


In [10]:
prueba_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

prueba_generator=prueba_datagen.flow_from_directory('./prueba/', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 
                                                 shuffle=True)


Found 4 images belonging to 1 classes.


In [11]:
print(model.predict(prueba_generator).round())

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [75]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serializar los pesos a HDF5
model.save_weights("model.h5")
print("Modelo Guardado!")
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()


 
# Compilar modelo cargado y listo para usar.
#loaded_model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

Modelo Guardado!
